# RNN Basic
- 기존 NN과 RNN의 차이점은 output이 다음의 cell로 연결된다.
- 어떻게 구현하지?
    - 1. cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden_size)
        - 출력의 hidden_size를 정해 준다.
    - 2. outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)
- 위와 같이 두단계로 구분한 이유
    - cell을 생성하는 부분을 구현부와 분리시켜, 적용 function을 자유롭게 변경 가능하다.

# Hi Hello Training
- 문자 하나를 입력하면 다른 문자를 예측하는 알고리즘

In [14]:
import tensorflow as tf
import numpy as np

# data create
idx2char = ['h', 'i', 'e', 'l', 'o']
x_data = [[0, 1, 0, 2, 3, 3]] # hihell
x_one_hot = [[[1, 0, 0, 0, 0], # h
              [0, 1, 0, 0, 0], # i
              [1, 0, 0, 0, 0], # h
              [0, 0, 1, 0, 0], # e
              [0, 0, 0, 1, 0], # l
              [0, 0, 0, 1, 0]]]# l

y_data = [[1, 0, 2, 3, 3, 4]] # ihello
sequence_length = 6
input_dim = 5
X = tf.placeholder(tf.int32, [None, sequence_length]) # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length]) # Y Label

# input_data's shape = [1, 6, 5]
hidden_size = 5
batch_size = 1
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True, reuse=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _state = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

# Cost: sequence loss
# output size = [batch_size, sequence_length]
# y_data = tf.constant([[1, 1, 1]])
# prediction = tf.constant([[[0.2, 0.7], [0.6, 0.2], [0.2, 0.9]]], dtype=tf.float32)
# weights = tf.constant([[1, 1, 1]], dtype=tf.float32)
# sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     print("loss: ", sequence_loss.eval())
    
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss: ", l, "prediction: ", result, "true Y: ", y_data)
        
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("/tPrediction str: ", ''.join(result_str))

ValueError: Dimension must be 2 but is 3 for 'transpose_12' (op: 'Transpose') with input shapes: [?,6], [3].